In [14]:
import pandas as pd
import gzip
from xgboost import XGBRegressor, XGBClassifier
from scipy.stats.mstats import winsorize

In [15]:
def get_data() -> pd.DataFrame:
    dfs = []

    for i in range(1, 11):
        base_url = f"dataset/raw/capstone.{i}.jsonl.gz"

        with gzip.open(base_url.format(str(i)), mode="r") as f:
            df_batch = pd.read_json(f, lines=True)
            dfs.append(df_batch)

    df = pd.concat(dfs, ignore_index=True)
    return df

In [16]:
df = get_data()

In [73]:
def insert_apps(df: pd.DataFrame):
    apps = set()
    for val in df["apps"]:
        for list_val in val:
            apps.add(list_val)

    for app in apps:
        df[f"is_using_{app}"] = 0

    for app in apps:
        df[f"is_using_{app}"] = df["apps"].apply(lambda x: 1 if app in x else 0)

    return df


In [74]:
def preprocess_service_type(df: pd.DataFrame) -> pd.DataFrame:
    service_type_mapping = {"Prepaid": 1, "Postpaid": 2, "Broadband": 3}
    df["service_type"] = df["service_type"].map(service_type_mapping)
    return df

In [75]:
def preprocess_churn(df: pd.DataFrame):
    churn_mapping = {False: 0, True: 1}
    df["churn"] = df["churn"].map(churn_mapping)
    return df


In [76]:
def impute_with_xgboost(df: pd.DataFrame, column_to_impute: str, job: str = "regression") -> pd.DataFrame:
    empty_df = df[df[column_to_impute].isna()].drop(columns=column_to_impute)

    X_train = df[df[column_to_impute].notna()].drop(columns=column_to_impute)
    y_train = df.loc[df[column_to_impute].notna(), column_to_impute]

    if job == "regression":
        xgb = XGBRegressor()
    elif job == "classification":
        xgb = XGBClassifier()
        
    model = xgb.fit(X_train, y_train)
    pred = model.predict(empty_df)
    df.loc[df[column_to_impute].isna(), column_to_impute] = pd.to_numeric(pred.flatten())
    
    return df

In [77]:
def winsorize_outliers(df: pd.DataFrame, iqr_multiplier: float = 1.5) -> pd.DataFrame:
    for column in df.select_dtypes(include=["number"]).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - iqr_multiplier * IQR
        upper_bound = Q3 + iqr_multiplier * IQR

        df[column] = winsorize(
            df[column],
            limits=[(df[column] < lower_bound).mean(), (df[column] > upper_bound).mean()],
        )

    return df

In [78]:
df = (
    df
    .pipe(insert_apps)
    .drop(columns=["id", "apps"])
    .drop_duplicates()
    .pipe(preprocess_service_type)
    .pipe(preprocess_churn)
    .pipe(impute_with_xgboost, column_to_impute="avg_call_duration")
    .pipe(impute_with_xgboost, column_to_impute="roaming_usage")
    .pipe(impute_with_xgboost, column_to_impute="auto_payment", job="classification")
    .pipe(impute_with_xgboost, column_to_impute="call_drops")
    .assign(**{
        "data_usage": lambda df: df["data_usage"].fillna(df["data_usage"].mean()),
        "tenure": lambda df: df["tenure"].fillna(df["tenure"].mean()),
        "monthly_charge": lambda df: df["monthly_charge"].fillna(df["monthly_charge"].median())
    })
    .pipe(winsorize_outliers)
    .astype({col: "int32" for col in df.select_dtypes("int64").columns})
    .astype({col: "float32" for col in df.select_dtypes("float64").columns})
 )

In [79]:
df

,age,tenure,service_type,avg_call_duration,data_usage,roaming_usage,monthly_charge,overdue_payments,auto_payment,avg_top_up_count,call_drops,customer_support_calls,satisfaction_score,churn,is_using_RitimGo,is_using_İzleGo,is_using_CüzdanX,is_using_Konuşalım,is_using_HızlıPazar
0,18,53.0,1,106.739998,139.720001,47.310001,59.450001,0,0.0,25,18.000000,13,1.34,0,0,0,0,0,0
1,26,15.0,1,31.549999,12.140000,21.520000,1221.650024,0,0.0,51,8.000000,3,2.57,0,0,0,0,0,0
2,32,152.0,2,30.639999,10.170000,31.110001,1170.449951,0,1.0,0,13.000000,10,8.02,0,0,0,0,0,0
3,33,35.0,1,60.446297,100.065697,36.029999,2025.170044,0,0.0,19,7.000000,11,5.96,0,0,0,0,0,0
4,18,243.0,1,85.620003,164.789993,46.400002,1518.189941,0,0.0,72,15.000000,6,8.29,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,33,113.0,2,45.549999,78.570000,10.940000,892.179993,2,1.0,0,4.000000,17,4.06,0,0,0,0,0,0
9999996,23,168.0,1,10.070000,185.910004,55.169998,2025.170044,0,0.0,64,7.000000,18,5.51,0,0,0,0,0,0
9999997,18,270.0,1,25.290001,43.180000,36.860001,1492.849976,0,1.0,23,3.000000,9,4.16,0,0,0,0,0,0
9999998,40,19.0,3,61.228397,23.660000,29.046535,310.239990,0,0.0,0,10.132260,6,6.70,0,0,0,0,0,0


In [82]:
df.to_csv("dataset/interim/cleaned_dataset.csv.gz", compression="gzip", index=False)